In [111]:
from src.utils import Utils
import torch
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

Utils().clean_corrupt_images(root_dir="images_files")

data_files = ImageFolder(root="images_files", transform=transform)

classes = data_files.classes

train_size = int(0.85 * len(data_files))
test_size = len(data_files) - train_size
train_set, test_set = random_split(data_files, [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [112]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

net = Net()
net.to(device)

def evaluate_model(net, data_loader, device):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


In [113]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(100):
    
    net.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')
   
    accuracy = evaluate_model(net, test_loader, device)
    print(f'Accuracy after epoch {epoch + 1}: {accuracy}%')

print('Finished Training')

Epoch 1, Loss: 0.6556366664099406
Accuracy after epoch 1: 63.49333333333333%
Epoch 2, Loss: 0.6107709813997688
Accuracy after epoch 2: 69.38666666666667%
Epoch 3, Loss: 0.5855728745550276
Accuracy after epoch 3: 69.6%
Epoch 4, Loss: 0.560185515557427
Accuracy after epoch 4: 71.09333333333333%
Epoch 5, Loss: 0.5334620462752968
Accuracy after epoch 5: 73.49333333333334%
Epoch 6, Loss: 0.5170289299574243
Accuracy after epoch 6: 75.33333333333333%
Epoch 7, Loss: 0.49920167565255996
Accuracy after epoch 7: 75.54666666666667%
Epoch 8, Loss: 0.48952974953565254
Accuracy after epoch 8: 75.41333333333333%
Epoch 9, Loss: 0.47738223069583074
Accuracy after epoch 9: 78.58666666666667%
Epoch 10, Loss: 0.4680342045893152
Accuracy after epoch 10: 78.16%
Epoch 11, Loss: 0.45939936973603374
Accuracy after epoch 11: 79.70666666666666%
Epoch 12, Loss: 0.4449987513607884
Accuracy after epoch 12: 78.8%
Epoch 13, Loss: 0.44025182515293
Accuracy after epoch 13: 80.24%
Epoch 14, Loss: 0.43031769806900655
Accu

In [114]:
torch.save(net.state_dict(), 'model_state_dict.pth')

In [109]:
from PIL import Image

image_path = 'images_files/Dog/55.jpg'

# Cargar la imagen
image = Image.open(image_path)  # Convertir a escala de grises si es necesario

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Aplicar transformaciones
image = transform(image)
image = image.unsqueeze(0)  # Añadir una dimensión de lote

image = image.to(device)

net.load_state_dict(torch.load("models/model.pth"))

net.eval()

with torch.no_grad():
    output = net(image)
    probabilities = F.softmax(output, dim=1)
    percentage = probabilities * 100

predicted_class = torch.argmax(probabilities, dim=1)
predicted_probability = probabilities[0, predicted_class.item()] * 100

print(f"Clase predicha: {predicted_class.item()}, Probabilidad: {predicted_probability.item():.2f}%")
print("Probabilidades en porcentaje de todas las clases:", percentage[0].tolist())

Clase predicha: 1, Probabilidad: 99.91%
Probabilidades en porcentaje de todas las clases: [0.0920671597123146, 99.9079360961914]
